In [1]:
# S3 prefix
prefix = "sm-docker-poc"

# Define IAM role
import boto3
import re
import os
import numpy as np
import pandas as pd
import io
import itertools
from sagemaker import get_execution_role
from sagemaker.predictor import csv_serializer
import sagemaker as sage
from time import gmtime, strftime

role = get_execution_role()

In [2]:
role

'arn:aws:iam::709891711940:role/service-role/AmazonSageMakerServiceCatalogProductsUseRole'

In [3]:
sess = sage.Session()

In [4]:
WORK_DIRECTORY = "sm_sklearn_docker/data"

data_location = sess.upload_data(WORK_DIRECTORY, key_prefix=prefix + "/data")

In [14]:
data_location

's3://sagemaker-us-east-1-709891711940/sm-docker-poc/data'

In [15]:
account = sess.boto_session.client("sts").get_caller_identity()["Account"]
region = sess.boto_session.region_name
# image = "709891711940.dkr.ecr.us-east-1.amazonaws.com/sm-poc:latest"
image = "709891711940.dkr.ecr.us-east-1.amazonaws.com/sm-poc1:latest"

In [16]:
account, region

('709891711940', 'us-east-1')

In [19]:
tree = sage.estimator.Estimator(
    image,
    role,
    1,
    "ml.c4.2xlarge",
    output_path="s3://{}/output".format(sess.default_bucket()),
    sagemaker_session=sess
)
tree.fit(data_location)

2022-10-07 13:30:24 Starting - Starting the training job...
2022-10-07 13:30:47 Starting - Preparing the instances for trainingProfilerReport-1665149424: InProgress
.........
2022-10-07 13:32:08 Downloading - Downloading input data...
2022-10-07 13:32:48 Training - Training image download completed. Training in progress..Starting the training.
Training complete.

2022-10-07 13:33:48 Uploading - Uploading generated training model
2022-10-07 13:33:48 Completed - Training job completed
Training seconds: 107
Billable seconds: 107


In [20]:
predictor = tree.deploy(1, "ml.m4.xlarge", serializer=csv_serializer)

-------!

In [48]:
a = [50 * i for i in range(3)]
b = [40 + i for i in range(10)]
indices = [i + j for i, j in itertools.product(a, b)]

df = pd.read_csv("data/iris.csv", header=None)
df.sample(3)
test_data = df.iloc[indices[:-1]]
test_data_x, test_data_y = test_data.iloc[:, 1:], test_data.iloc[:, 0]
data = io.BytesIO(test_data_x.to_csv(index=False).encode('utf-8'))
data.getvalue()

b'1,2,3,4\n5.0,3.5,1.3,0.3\n4.5,2.3,1.3,0.3\n4.4,3.2,1.3,0.2\n5.0,3.5,1.6,0.6\n5.1,3.8,1.9,0.4\n4.8,3.0,1.4,0.3\n5.1,3.8,1.6,0.2\n4.6,3.2,1.4,0.2\n5.3,3.7,1.5,0.2\n5.0,3.3,1.4,0.2\n5.5,2.6,4.4,1.2\n6.1,3.0,4.6,1.4\n5.8,2.6,4.0,1.2\n5.0,2.3,3.3,1.0\n5.6,2.7,4.2,1.3\n5.7,3.0,4.2,1.2\n5.7,2.9,4.2,1.3\n6.2,2.9,4.3,1.3\n5.1,2.5,3.0,1.1\n5.7,2.8,4.1,1.3\n6.7,3.1,5.6,2.4\n6.9,3.1,5.1,2.3\n5.8,2.7,5.1,1.9\n6.8,3.2,5.9,2.3\n6.7,3.3,5.7,2.5\n6.7,3.0,5.2,2.3\n6.3,2.5,5.0,1.9\n6.5,3.0,5.2,2.0\n6.2,3.4,5.4,2.3\n'

In [49]:
runtime = boto3.client("runtime.sagemaker")
endpoint_name = 'sm-poc1-2022-10-07-13-40-23-558'
response = runtime.invoke_endpoint(
    EndpointName=endpoint_name, ContentType="text/csv", Body=data
)
response["Body"].read().decode('utf-8')

/home/ec2-user/anaconda3/envs/amazonei_mxnet_p36/lib/python3.6/site-packages/boto3/compat.py:88: PythonDeprecationWarning: Boto3 will no longer support Python 3.6 starting May 30, 2022. To continue receiving service updates, bug fixes, and security updates please upgrade to Python 3.7 or later. More information can be found here: https://aws.amazon.com/blogs/developer/python-support-policy-updates-for-aws-sdks-and-tools/
  warnings.warn(warning, PythonDeprecationWarning)


'virginica\nsetosa\nsetosa\nsetosa\nsetosa\nsetosa\nsetosa\nsetosa\nsetosa\nsetosa\nsetosa\nversicolor\nversicolor\nversicolor\nversicolor\nversicolor\nversicolor\nversicolor\nversicolor\nversicolor\nversicolor\nvirginica\nvirginica\nvirginica\nvirginica\nvirginica\nvirginica\nvirginica\nvirginica\nvirginica\n'